a bs4 and requests excercise to scrape all the results from [City2Surf](http://city2surf.com.au/past-results/) and put them into a pandas df, then do some data analysis.

_work in progress_

In [171]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

from IPython.core.display import HTML

## links to all the past result pages

Now to go through each years link in the past results page, then click on results, scrape result, and keep clicking more until all results are scraped.

In [67]:
r = requests.get("http://city2surf.com.au/past-results/")

In [98]:
soup = BeautifulSoup(r.text, "lxml")

In [139]:
# get all the links
all_links = soup.find_all("a")

# we only want the results links
links = [link["href"] for link in all_links if "https://secure.tiktok.biz/results/city2surf/" in str(link)]

# the 2017 url isn't listed, so inserting manually
links.insert(0, "https://secure.tiktok.biz/results/city2surf/2017")

for link in links:
    print(link)

https://secure.tiktok.biz/results/city2surf/2017
https://secure.tiktok.biz/results/city2surf/2016
https://secure.tiktok.biz/results/city2surf/2015/
https://secure.tiktok.biz/results/city2surf/2014/
https://secure.tiktok.biz/results/city2surf/2013/
https://secure.tiktok.biz/results/city2surf/2012/
https://secure.tiktok.biz/results/city2surf/2011/
https://secure.tiktok.biz/results/city2surf/2010/
https://secure.tiktok.biz/results/city2surf/2009/
https://secure.tiktok.biz/results/city2surf/2008/
https://secure.tiktok.biz/results/city2surf/2007/
https://secure.tiktok.biz/results/city2surf/2006
https://secure.tiktok.biz/results/city2surf/2005/
https://secure.tiktok.biz/results/city2surf/2004/
https://secure.tiktok.biz/results/city2surf/2003/
https://secure.tiktok.biz/results/city2surf/2002/
https://secure.tiktok.biz/results/city2surf/2001/
https://secure.tiktok.biz/results/city2surf/2000/


All the links look good, but each of those links further links to the actual results. So lets take a look at an example results page:

In [144]:
soup = BeautifulSoup(requests.get("https://secure.tiktok.biz/results/city2surf/2003/").content, "lxml")
HTML(str(soup.find("table")))

Race,Entrants,Finishers,1st Male,1st Fml.,Avg Time,
The Sun-Herald City2Surf,"58,444","48,251",00:41:55,00:47:28,01:57:02,results


So now to go through all the above links and get the link we want, which just happens to the best first link of the first table on the results page:

In [190]:
results_links = []
for link in links:
    soup = BeautifulSoup(requests.get(link).content, "lxml")
    table = soup.find("table")
    url = [l["href"] for l in table.find_all("a")][0]
    # remove "../" patterns from url
    url = url.replace("../", "")
    results_links.append("https://secure.tiktok.biz/results/" + url)

In [191]:
results_links

['https://secure.tiktok.biz/results/list.aspx?eventid=city2surf&edition=2017&race=14KM',
 'https://secure.tiktok.biz/results/list.aspx?eventid=city2surf&edition=2016&race=14KM',
 'https://secure.tiktok.biz/results/list.aspx?eventid=city2surf&edition=2015&race=14KM',
 'https://secure.tiktok.biz/results/list.aspx?eventid=city2surf&edition=2014&race=14KM',
 'https://secure.tiktok.biz/results/list.aspx?eventid=city2surf&edition=2013&race=C2S',
 'https://secure.tiktok.biz/results/list.aspx?eventid=city2surf&edition=2012&race=C2S',
 'https://secure.tiktok.biz/results/list.aspx?eventid=city2surf&edition=2011&race=C2S',
 'https://secure.tiktok.biz/results/list.aspx?eventid=city2surf&edition=2010&race=C2S',
 'https://secure.tiktok.biz/results/list.aspx?eventid=city2surf&edition=2009&race=C2S',
 'https://secure.tiktok.biz/results/list.aspx?eventid=city2surf&edition=2008&race=14KM',
 'https://secure.tiktok.biz/results/list.aspx?eventid=city2surf&edition=2007&race=14KM',
 'https://secure.tiktok.bi

now to download all the results into a pandas table. do i end up making one giant table or a table for each year?

# get results

First, lets see how the results are displayed on a random result page:

In [194]:
url = results_links[np.random.randint(0, len(results_links))]
r = requests.get(url)
soup = BeautifulSoup(r.text, "lxml")

In [195]:
table = soup.find("table")
HTML(str(table))

#,Bib,Name,Finish Time,Net Time,Place,Gender,G/Pl,Division,D/Pl,
1,00064,MICHAEL SHELLEY,00:41:02,00:40:57,1,M,1,M25-29,1,view
2,00059,BEN ST LAWRENCE,00:41:12,00:41:07,2,M,2,M25-29,2,view
3,00067,CLINT PERRETT,00:41:52,00:41:47,3,M,3,M25-29,3,view
4,00086,CHRIS HAMER,00:42:22,00:42:17,4,M,4,M19-24,1,view
5,00054,ERWIN MCRAE,00:42:29,00:42:24,5,M,5,M30-34,1,view
6,00017,GRANT PAGE,00:42:36,00:42:31,6,M,6,M25-29,4,view
7,00081,HARRY SUMMERS,00:42:41,00:42:35,7,M,7,M19-24,2,view
8,00046,ANTHONY HABER,00:42:50,00:42:43,8,M,8,M30-34,2,view
9,00005,THOMAS CRASTI,00:43:35,00:43:29,9,M,9,M19-24,3,view
10,00023,CHRIS HARTLEY,00:43:36,00:43:29,10,M,10,M25-29,5,view


So the first page of results lists a hundred entries, which we can read this into a dataframe:

In [199]:
df = pd.read_html(table.prettify(), header=0)[0]
print(df.shape)
df.head()

(100, 11)


,#,Bib,Name,Finish Time,Net Time,Place,Gender,G/Pl,Division,D/Pl,Unnamed: 10
0,1,64,MICHAEL SHELLEY,00:41:02,00:40:57,1,M,1,M25-29,1,view
1,2,59,BEN ST LAWRENCE,00:41:12,00:41:07,2,M,2,M25-29,2,view
2,3,67,CLINT PERRETT,00:41:52,00:41:47,3,M,3,M25-29,3,view
3,4,86,CHRIS HAMER,00:42:22,00:42:17,4,M,4,M19-24,1,view
4,5,54,ERWIN MCRAE,00:42:29,00:42:24,5,M,5,M30-34,1,view


Now, how do I paginate through the results so I get all of them?

Now, how do I paginate through the results so I get all of them? the results are displayed in a javascripty thing, and the next button is a javascript button which loads the next 100 results on the same page.